# Exercise 2: Temporal Hello World — Solution

Complete implementation of the Temporal Hello World workflow with an activity.

## Architecture Pattern 

```
Workflow Execution Request 
    ↓
Temporal Workflow (orchestration) 
    ↓
Temporal Activity (unit of work) 
    ↓
[Process data, call APIs, etc.]
    ↓
Activity completes [OK]
    ↓
Workflow returns result 
    ↓
Return to caller [OK]
```

**Key Benefits:**
- [OK] Activities automatically retry on failure
- [OK] Workflow state persists across crashes
- [OK] Full execution history in Temporal UI

## Setup

Before running this solution, ensure you have:

1. **Temporal Server Running:**
   - Follow `temporal_installation.ipynb` in the project root


In [ ]:
import asyncio
from datetime import datetime, timedelta

import nest_asyncio
import pytz
from temporalio import activity, workflow
from temporalio.client import Client
from temporalio.worker import UnsandboxedWorkflowRunner, Worker

## 1. Define an `activity.py`

In [ ]:
@activity.defn
async def process_data(data: str) -> str:
    """Activity that simulates work by upper-casing the payload."""
    activity.logger.info(" Activity started")
    await asyncio.sleep(5)  # Simulate some processing delay
    result = f"Processed: {data.upper()}"
    activity.logger.info("[OK] Activity completed")
    return result


## 2. Define a `workflow.py`

In [ ]:
@workflow.defn
class HelloWorkflowTemporal:
    """Workflow that orchestrates the activity call."""

    @workflow.run
    async def run(self, name: str) -> str:
        workflow.logger.info(" Workflow started")
        result = await workflow.execute_activity(
            process_data,
            args=[f"Hello {name}"],
            start_to_close_timeout=timedelta(seconds=10),
        )
        workflow.logger.info("[OK] Workflow finished")
        return f"Workflow result: {result}"

## Create `worker.py`

In [ ]:
async def run_worker():  # Define async function to start and run the worker
    """Start a Temporal worker that listens for workflow and activity tasks."""
    # Connect to local Temporal server
    client = await Client.connect(
        "localhost:7233",  # Temporal server address
    )

    # Create worker that polls the task queue for work
    task_queue = "hello-temporal-task-queue"
    worker = Worker(
        client,  # Use the connected Temporal client
        task_queue=task_queue   ,  # Which queue to poll for tasks
        workflows=[HelloWorkflowTemporal],  # sets the workflow type name in UI
        activities=[process_data],  # List of activities this worker can execute
        workflow_runner=UnsandboxedWorkflowRunner(),
    )

    print(f"[OK] Worker started on task queue: {task_queue}")
    print("   Listening for workflow and activity tasks...")
    # Start polling and executing tasks (blocks until stopped)
    await worker.run()


# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()
worker_task = asyncio.create_task(run_worker())
print(" Worker running in background")


## Create `starter.py`

Use the cell below to run the fully implemented solution script.

In [ ]:
async def run_solution():  # Define async function to execute the workflow
    """Execute Temporal workflow."""

    # Generate workflow ID with EST timestamp for human-readable tracking
    est = pytz.timezone("US/Eastern")  # Create EST timezone object
    now = datetime.now(est)  # Get current time in EST
    # Format timestamp as readable string with day-month-date-time pattern
    workflow_id = f"weather-{now.strftime('%a-%b-%d-%I%M%S').lower()}est"

    # Connect to Temporal server
    client = await Client.connect(
        "localhost:7233"  # Enable OpenAI Agents integration
    )

    print(f" Starting workflow: {workflow_id}")

    # Start the workflow (non-blocking) and get handle for tracking
    task_queue = "hello-temporal-task-queue"
    handle = await client.start_workflow(
        HelloWorkflowTemporal.run,  # Workflow method to execute
        "Solution02",
        id=workflow_id,  # Unique workflow ID for tracking in Temporal UI
        task_queue=task_queue,  # Queue where worker will pick up this workflow
    )

    print(f"[OK] Workflow started: {handle.id}")
    # Print Temporal UI link for observing workflow execution
    print(
        f"🔗 View in Temporal UI: http://localhost:8233/namespaces/default/workflows/{workflow_id}\n"
    )
    # Wait for workflow to complete and get result (blocking)
    result = await handle.result()
    print(f" Workflow completed with result: {result}")

# Run the solution with Jupyter-specific async handling
try:
    # Try to get existing event loop (Jupyter has one running)
    loop = asyncio.get_running_loop()
    # Execute in existing loop
    await run_solution()
except RuntimeError:
    # If no loop exists, create new one and run
    asyncio.run(run_solution())
